# Gradiente e Classificação multinomial

Nícolas Hecker Silva RA 186132


## Descrição 

Esta tarefa explora conceitos de c ́alculo vetorial, otimiza ̧c ̃ao e m ́etodos de gradiente descendente no contexto de aprendizado de m ́aquina, com foco em classifica ̧c ̃ao multinomial.

## Etapa 1: Funções Convexas e Otimização

Dado uma função $f(x) = x^TAx + b^Tx + c$ faça três versões mudando a matriz A para que a função seja:


- Fortemente Convexa.
- Convexa.
- Não Convexa.

Evidencia cada uma das propriedades.


Dado o enunciado acima, como não foram especificados os tipos das variáveis e constantes será assumido que:
x: É um vetor coluna de dimensão n (nx1)
A: É uma matriz quadrada de dimensão nxn
b: É um vetor coluna de dimensão n (nx1)
c: É um escalar

Dessa forma a dimensão das parcelas será:

$x^TAx = (nx1)^T * (nxn) * (nx1) = (1xn) * (nxn) * (nx1) = (1)$

$b^Tx = (nx1)^T * (nx1) = (1xn) * (nx1) = (1)$

$c = (1)$

Dessa forma f(x) retornará um valor real.

Uma função $ f: \mathbb{R}^n \to \mathbb{R} $ é **fortemente convexa** com parâmetro $ m > 0 $ se, para todo $ x, y \in \mathbb{R}^n $, satisfizer:

$
f(x) \geq f(y) + \nabla f(y)^T (x - y) + \frac{m}{2} \| x - y \|^2.
$

Se $ f(x) = x^T A x + b^T x + c $, para que $ f $ seja fortemente convexa, a matriz $ A $ deve ser **positiva definida** com todos os autovalores maiores ou iguais a $ m $, ou seja, $ \lambda_{\min}(A) \geq m > 0 $.

Por fim, basta que o menor autovalor de $A$ seja maior que 0, sendo (novamente) **positiva definida**. E para isso, basta ela ser simétrica. Uma matriz exemplo seria

$$
A = \begin{pmatrix}
  2 & 0 \\
  0 & 3 
\end{pmatrix}
$$

cujos autovalores são exatamente 2 e 3.

Já para a função ser **convexa** basta que seus autovalores sejam não negativos, ou seja, eles podem ser nulos. Dessa forma, a matriz exemplo

$$
A = \begin{pmatrix}
  2 & 0 \\
  0 & 0
\end{pmatrix}
$$

terá os autovalores 0 e 2, fazendo com que f(x) seja **convexa**.

Dessa forma, para que a função f seja **não convexa**, é necessário que algum autovalor seja negativo, pois nesse caso a função f(x) não terá curvatura positiva em todas as direções, podendo curvar-se para baixo em algumas direções, o que viola a convexidade.

Um exemplo é a matriz

$$
A = \begin{pmatrix}
  2 & 0 \\
  0 & -1
\end{pmatrix}
$$

em que os autovalores são 2 e -1, sendo -1 um valor negativo fazendo com que A não seja semidefinida positiva.



In [7]:
!pip install kagglehub

/bin/bash: linha 1: /home/nicolas/Programas/AlgeLin-ML/.venv/bin/pip: não é possível executar: arquivo necessário não encontrado


In [6]:
!pip install kagglehub

import kagglehub

# Download latest version
path = kagglehub.dataset_download("hojjatk/mnist-dataset")

print("Path to dataset files:", path)

/bin/bash: linha 1: /home/nicolas/Programas/AlgeLin-ML/.venv/bin/pip: não é possível executar: arquivo necessário não encontrado


ModuleNotFoundError: No module named 'kagglehub'